# NameClassifier　チュートリアル

このノートでは、NameClassifierモジュールの基本的な使い方を説明していきます。
NameClassifierモジュールはpythonのクラスファイルで、個人の名前に基づき日本人か外国人かを判断します。このバージョンではベイズ分類器とBag of Wordsモデルが用いられています。<br>
ここでは
- データを読み込み、学習用とテスト用に分ける 
- データをベクトル化し、モデルに学習させる
- テスト用データでモデルを評価、また新しいデータで予測してみる
- 学習後のモデルを保存する
- 学習済みのモデルを読み込んで使う


## 必要なライブラリ
このモジュールはいくつかのPythonライブラリを使用しており、使う前にこれらがインストールされていることを確認してください。

- [scikit learn](https://scikit-learn.org/stable/)
- [pandas](https://pandas.pydata.org/)
- [pickle](https://docs.python.org/3/library/pickle.html)
また上のライブラリが依存しているパッケージもインストールしておいてください。

## セットアップ
ライブラリがインストールされていれば、あとは`model.py`を使うダイレクトリにおいて置くだけで他のスクリプトから呼び出したりして使用可能です。
また学習データや保存したモデルを使う場合は同じダイレクトリ内に入れておいたほうが良いでしょう。
このチュートリアルでは、
- プロジェクトフォルダ
    - model.py: クラスファイル
    - data: データフォルダ
        - jp_names.cvs: 日本名データ
        - f_names.csv: 外国名データ
    - model: 学習モデルフォルダ

というフォルダ構造の前提となります。
それでは始めましょう！

## データを読み込む
ここではCSVファイルからデータを読み込み、そしてデータを任意の割合で学習用とテスト用に分割します。
データはCSV形式で、下のようなフォーマットの必要があります。


>Country, Address, name, other col..<br>
value1, value2, 鈴木　一郎, value4<br>
value1, value2, 鈴木　二郎, value4

最初の行にカラム名があり、下にデータが続いています。カラム名は名前のデータだけ'name'であれば他のカラムは影響ありません。また名前は国籍に関わらず姓名の間がスペースで区切られている必要があります。
それでは`load_data()`メソッドを用いてデータを読み込みます。

In [13]:
## モジュールをインポート
from model import NameClassifier

# このメソッドは x_train, x_test, y_train, y_test をこの順番で返します。
x_train, x_test, y_train, y_test = NameClassifier.load_data('data/jp_names.csv', 'data/f_names.csv', test_size=0.4)

print('名前データ　 type:{}\n{}'.format(type(x_train), x_train.sample(10)))
print('\nラベル: 　{}, type: {}'.format(y_train, type(y_train)))

名前データ　 type:<class 'pandas.core.series.Series'>
98583           樊 柏翰
47855          高橋 翔太
94555         中津川 七夏
72841    Aneta Dobre
57767           坂本 零
15022           渚 七夏
32917          井高 英樹
25096         近藤 裕美子
95669           宮沢 淳
25372           近藤 淳
Name: name, dtype: object

ラベル: 　[0 1 0 ... 1 1 0], type: <class 'numpy.ndarray'>


それぞれ
- x_train: 学習用名前データ 
- x_test:　テスト用名前データ
- y_train:　学習用ラベル
- y_test:　テスト用ラベル

で、PandasのSeriesタイプとNumpyのndarrayタイプです。
- `test_size`でテスト用データの割合を0~1の間の小数で指定できます。
- ファイル名はこのクラスファイルの場所からのパス＋ファイル名です。

## 学習！

それでは準備が出来たところでモデルに学習させてみましょう。まずはNameClassifierクラスのインスタンスを作成し、
そこに学習用データを入力します。

`.train(x_train, y_train)`メソッドを使用します。

In [7]:
# インスタンスをclfとして作成
clf = NameClassifier()
# 学習データを入力し、学習させる。
clf.train(x_train, y_train)

Fitting the vectorizer and training the model...
training completed!


## モデル評価

ここでは先程作成したテスト用データを用いてモデルを評価します。分類問題なので評価指標は
- 正解率(Accuracy): 予測したデータのうちどれくらいが正解だったか（国籍に関わらず）
- 適合率(Precision): 日本名と予測した名前のうち、どれくらいが実際に日本名だったか
- 再現率(Recall): 実際の日本名のうち、どれくらいを日本名として予測できたか

このモジュールでは、`.evaluate(x_test, y_test)`にテスト用データとラベルを入力することで評価出来ます。

In [10]:
# モデル評価、３つの値を含んだディクショナリを返す。
model_eval = clf.evaluate(x_test, y_test)
st = '正解率：{}%\n適合率：{}\n再現率：{}'.format(model_eval['accuracy']*100, model_eval['precision'], model_eval['recall'])
print(st)

正解率：99.79124999999999%
適合率：1.0
再現率：0.9958322934863988


* *学習済みの名前に関しては分類精度は高いですが、未知のデータ、名前は明らかに日本人名でも外国人判定されるようです。


## モデルを保存する

このモジュールはモデル/クラスをPythonのPickleファイルとして保存します。ファイル名を指定する際は拡張子が`.pickle`になっているかを確認してください。
`.save_model()`を使い、

In [11]:
# モデルを保存、ファイルパス、ファイル名、拡張子を忘れずに。
clf.save_model('model/trained_model.pickle')

## 学習済みのモデルを使う

上で保存したモデルを読み込み、先程のテストデータを用いて評価、また予測してみましょう。

予測する際は`.predict()`を使い、名前データをリストかPandasのSeriesとして入力し、出力は
- 1: 日本名と予測
- 0: それ以外と予測

をリストとして返します。

In [12]:
# モデルを読み込む。
trained_clf = NameClassifier.load_model('model/trained_model.pickle')
# モデルを評価
model_eval = trained_clf.evaluate(x_test, y_test)
st = '正解率：{}%\n適合率：{}\n再現率：{}'.format(model_eval['accuracy']*100, model_eval['precision'], model_eval['recall'])
print(st)

# 他の名前で予測してみる
names = ['John Wick', '문　재인', '佐藤　たける']
print(trained_clf.predict(names))

loading the model
正解率：99.79124999999999%
適合率：1.0
再現率：0.9958322934863988
[0, 0, 1]
